<a href="https://colab.research.google.com/github/viett887g/VSM_20230311/blob/main/SVM_20230311.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# **I. Chọn dataset :USA Housing**

In [ ]:
def readdata(file):
    data = pd.read_csv(file)
    del data['Address']
    data = np.array(data)
    data = preprocessing.MinMaxScaler().fit_transform(data)
    X = data[:,:-1]
    y = data[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

In [ ]:
def svm_cost(theta, X, y, C):
    m = X.shape[0]
    hinge_loss = 1 - y * np.dot(X, theta)
    hinge_loss[hinge_loss < 0] = 0
    cost = 0.5 * np.dot(theta, theta) + (C * np.sum(hinge_loss)) / m
    return cost

def svm_gradient(theta, X, y, C):
    m = X.shape[0]
    pred = np.dot(X, theta)
    pred[pred >= 0] = 1
    pred[pred < 0] = -1
    gradient = theta - (C * np.dot(X.T, y * pred)) / m
    return gradient

# II. Chọn 2 phương pháp **Gradient Descent (GD)**, **Momentum Based GD** tối ưu để tối ưu hoá loss **SVM** cho tập dữ liệu USA Housing

##**2.1 Gradient Descent (GD)**

In [ ]:
def svm_train(X, y, C, learning_rate, num_iterations):
    m, n = X.shape
    theta = np.zeros(n)

    for i in range(num_iterations):
        cost = svm_cost(theta, X, y, C)
        gradient = svm_gradient(theta, X, y, C)
        theta -= learning_rate * gradient

        if i % 100 == 0:
            # print("Iteration:", i, "Cost:", cost)
            pass

## **2.2 Momentum Based GD**

In [ ]:
def svm_train_momentum(X, y, C, learning_rate, num_iterations, momentum):
    m, n = X.shape
    theta = np.zeros(n)
    velocity = np.zeros(n)

    for i in range(num_iterations):
        cost = svm_cost(theta, X, y, C)
        gradient = svm_gradient(theta, X, y, C)

        velocity = momentum * velocity + learning_rate * gradient
        theta -= velocity

        if i % 100 == 0:
            # print("Iteration:", i, "Cost:", cost)
            pass

    return theta

# **III. So sánh hiệu suất của các thuật toán**

In [ ]:
def evaluate_performance(X_test, y_test, w):
    # Tính toán dự đoán trên tập test
    y_pred = np.dot(X_test, w)

    # Tính toán Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)

    # Tính toán Rmse
    rmse = np.sqrt(mse)


    # Trả về kết quả hiệu suất của mô hình
    return mse, rmse

In [ ]:
def main():
    # Đọc dữ liệu từ file
    X_train, X_test, y_train, y_test = readdata("/content/drive/MyDrive/USA_Housing.csv")

    # Khởi tạo các tham số huấn luyện
    C = 1.0
    learning_rate = 0.01
    num_iterations = 1000
    momentum = 0.9
    # Huấn luyện mô hình SVM
    theta = svm_train(X_train, y_train, C, learning_rate, num_iterations)
    mse, rmse=evaluate_performance(X_test, y_test, theta)
    # Tính giá trị loss trên tập test
    test_cost = svm_cost(theta, X_test, y_test, C)
    print("Giá trị loss tối ưu trên tập test của thuật toán gradient descent :", test_cost)
    print("Mean Squared Error của Gradient Descents (MSE):", mse)
    print('Root Mean Squared Error của Gradient Descents:',rmse)
    print('\n','\n')

    theta=svm_train_momentum(X_train, y_train, C, learning_rate, num_iterations, momentum)
    test_cost = svm_cost(theta, X_test, y_test, C)
    print("Giá trị loss tối ưu trên tập test của thuật toán Momentum Based Gradient Descent :", test_cost)
    mse, rmse=evaluate_performance(X_test, y_test, theta)
    print("Mean Squared Error (MSE) của momentum_based_gradient_descent:", mse)
    print('Root Mean Squared Error của momentum_based_gradient_descent:',rmse)
if __name__ == "__main__":
    main()


Giá trị loss tối ưu trên tập test của thuật toán gradient descent : 0.8302422295015303
Mean Squared Error của Gradient Descents (MSE): 0.03207110809670577
Root Mean Squared Error của Gradient Descents: 0.1790840810812222

 

Giá trị loss tối ưu trên tập test của thuật toán Momentum Based Gradient Descent : 0.8302420371372262
Mean Squared Error (MSE) của momentum_based_gradient_descent: 0.0320794725113606
Root Mean Squared Error của momentum_based_gradient_descent: 0.17910743287580388


# **IV.Nhận xét và đánh giá**

**Nhận xét:**

giá trị loss tối ưu trên tập test của cả hai thuật toán không có sự khác biệt lớn (0.8302422295015303 cho GD và 0.8302420371372262 cho Momentum GD). Tuy nhiên, MSE và RMSE của Momentum GD (0.0320794725113606 và 0.17910743287580388) có một ít lớn hơn so với MSE và RMSE của GD (0.03207110809670577 và 0.1790840810812222).

**Kết luận :**

Dựa trên kết quả này, không có sự khác biệt rõ ràng giữa hai thuật toán trong trường hợp cụ thể này. Tuy nhiên, Momentum GD có thể giúp giảm sự dao động và tăng tính ổn định trong quá trình tối ưu hóa.

# **V. Ưu nhược điểm của phương pháp tối ưu.**

**Ưu điểm của phương pháp tối ưu Gradient Descent (GD):**

Đơn giản và dễ hiểu: Gradient Descent là một phương pháp tối ưu đơn giản và dễ hiểu. Nó chỉ yêu cầu tính đạo hàm đơn giản của hàm mất mát, và cập nhật các tham số theo hướng giảm độ dốc của đạo hàm.

Hiệu quả tính toán: GD thường được sử dụng trong nhiều mô hình học máy công nghệ cao, bởi vì nó có thể xử lý tập dữ liệu lớn một cách hiệu quả. Việc tính toán gradient chỉ phụ thuộc vào kích thước của batch, và không yêu cầu tính toán ma trận đầy đủ.

Hội tụ tới điểm tối ưu cục bộ: Nếu được cài đặt đúng, GD có khả năng hội tụ tới điểm tối ưu cục bộ. Nếu hàm mất mát là lồi và tốt hơn, GD có thể cung cấp giải pháp tối ưu đối tượng.

**Nhược điểm của phương pháp tối ưu Gradient Descent (GD):**

Tốc độ hội tụ chậm: GD có thể hội tụ chậm đối với một số bài toán, đặc biệt là khi hàm mất mát có đặc trưng vùng trong lỏng lẻo hoặc bề mặt lõm. Việc cập nhật các tham số trên toàn bộ dữ liệu có thể cần nhiều vòng lặp để đạt điểm tối ưu.
Momentum Based Gradient Descent (GD với đà):


**Ưu điểm Momentum Based GD** có thể giúp cải thiện tốc độ hội tụ của GD. Bằng cách tích lũy đà từ các bước trước, nó có khả năng vượt qua các mục tiêu hẹp và giảm thiểu trỗi dạng cục bộ, đồng thời giảm ảnh hưởng của các điểm nhiễu trên đường đi hội tụ.

**Nhược điểm Momentum Based GD**: Mặc dù hiệu quả hơn so với GD cơ bản, tuy nhiên không phải lúc nào Momentum Based GD cũng tốt hơn. Trong một số trường hợp, điều này có thể gây ra quá đà, kéo dài thời gian hội tụ hoặc không hội tụ tới điểm tối ưu. Bên cạnh đó, việc điều chỉnh thông số đà có thể không dễ dàng và yêu cầu thử nghiệm.